# Chapter 13 — Model Versioning and Aliases

This notebook demonstrates how to use aliases as stable pointers for
deployment pipelines. Aliases let us promote models without changing
deployment configuration.

**Continues from:** `01_model_packaging` (model registered in Unity Catalog)

In [0]:
%pip install mlflow
%restart_python

## 1. Set Up Variables

These should match the catalog and schema used during model registration.

In [0]:
CATALOG = "demo"
SCHEMA  = "finance"

## 2. Assign a Model Alias

Aliases act as stable pointers that deployment configurations reference.
Common aliases include "champion" (current production), "challenger"
(candidate under evaluation), and "archived" (retired version).

In [0]:
from mlflow import MlflowClient

client = MlflowClient()

model_name = f"{CATALOG}.{SCHEMA}.data_quality_agent"

# Point the "champion" alias at version 4
client.set_registered_model_alias(
    name=model_name,
    alias="champion",
    version=4
)

print(f"Alias 'champion' now points to version 4 of {model_name}")

## 3. DAB Configuration — Alias-Based Job

In a DAB configuration, we reference the alias rather than a version number.
The deployment configuration stays the same across environments.

```yaml
resources:
  jobs:
    inference_job:
      name: "data-quality-agent-inference"
      tasks:
        - task_key: run_inference
          notebook_task:
            notebook_path: ./notebooks/run_agent.py
            base_parameters:
              model_name: "${var.catalog}.${var.schema}.data_quality_agent@champion"
```

The only thing that changes between development and production is the value
of the variables, not the structure of the configuration.